In [1]:
import azureml.core
from azureml.core.authentication import InteractiveLoginAuthentication
from azureml.core import Experiment, Workspace
import azureml.core
from azureml.core.authentication import InteractiveLoginAuthentication
from azureml.core import Experiment, Workspace

# Check core SDK version number
print("You are currently using version", azureml.core.VERSION, "of the Azure ML SDK")
print("")

# Log In to Azure ML Workspace
interactive_auth = InteractiveLoginAuthentication(tenant_id="76f90eb1-fb9a-4446-9875-4d323d6455ad")

ws = Workspace.from_config(auth=interactive_auth)
print('Workspace name: ' + ws.name, sep='\n')
from azureml.core import Dataset

# get the dataset from Azure ML Workspace
aml_dataset = Dataset.get_by_name(ws, 'train_ds', version='latest')
pdf = aml_dataset.to_pandas_dataframe()

You are currently using version 1.5.0 of the Azure ML SDK

Workspace name: team15


In [112]:
import lightgbm
import pandas as pd
import numpy as np
from lightgbm import LGBMClassifier
from sklearn.model_selection import KFold
from tqdm.notebook import tqdm_notebook
import gc
from xgboost import XGBClassifier

In [3]:
def custom_metric(answers, take_top_ratio=0.25):
    answers.sort_values(by='uplift', inplace=True, ascending=False)
    n_samples = int(np.ceil(answers.shape[0] * take_top_ratio))
    answers = answers.iloc[:n_samples, :]
    answers_test = answers[answers['group'] == 'test']['response_att'].sum() / \
                   answers[answers['group'] == 'test'].shape[0]
    answers_control = answers[answers['group'] == 'control']['response_att'].sum() / \
                      answers[answers['group'] == 'control'].shape[0]
    return (answers_test - answers_control) * 100

In [4]:
pdf.group = pdf.group.map({'test': 1, 'control': 0})
pdf.gender = pdf.gender.fillna('Не определен')
y = pdf.group == pdf.response_att
pdf = pd.concat([pdf.drop('gender', axis=1), pd.get_dummies(pdf['gender'])], axis=1)
train = pdf.drop(['group', 'response_att'], axis=1)
train = train.rename({'М': 'gender_m', 'Ж': 'gender_f', 'Не определен': 'gender_ndf'}, axis=1)
train = train.set_index('CardHolder')

In [5]:
train.shape

(687029, 195)

In [6]:
train.isna().sum().sort_values().tail(10)

k_var_disc_share_1m_g49           414369
k_var_count_per_cheque_1m_g49     414473
k_var_sku_price_1m_g49            414473
k_var_disc_share_15d_g24          442054
k_var_count_per_cheque_15d_g24    442121
k_var_disc_share_15d_g34          468467
k_var_count_per_cheque_15d_g34    468551
k_var_sku_price_15d_g34           468551
k_var_disc_share_15d_g49          496159
k_var_sku_price_15d_g49           496259
dtype: int64

In [7]:
496259/687029

0.7223261317935633

In [8]:
(train.isna().sum() > train.shape[0] / 2).sum()

27

In [9]:
index1 = train.isna().sum() < train.shape[0] / 2
train = train.loc[:, index1]

In [10]:
train.shape

(687029, 168)

In [11]:
model = LGBMClassifier(class_weight='balanced',  n_jobs=- 1, silent=True, n_estimators=35, max_depth=2)

In [12]:
model.fit(train, y)

LGBMClassifier(boosting_type='gbdt', class_weight='balanced',
        colsample_bytree=1.0, importance_type='split', learning_rate=0.1,
        max_depth=2, min_child_samples=20, min_child_weight=0.001,
        min_split_gain=0.0, n_estimators=35, n_jobs=-1, num_leaves=31,
        objective=None, random_state=None, reg_alpha=0.0, reg_lambda=0.0,
        silent=True, subsample=1.0, subsample_for_bin=200000,
        subsample_freq=0)

In [13]:
feature_importances = pd.DataFrame({
    'features': train.columns,
    'feature_importances': model.feature_importances_}
            ).sort_values(by='feature_importances')
feature_importances.head()

,features,feature_importances
0,age,0
103,k_var_sku_price_3m_g32,0
104,k_var_sku_price_3m_g33,0
105,k_var_sku_price_3m_g34,0
106,k_var_sku_price_3m_g40,0


In [14]:
feature_importances.tail()

,features,feature_importances
69,k_var_days_between_visits_15d,6
70,k_var_days_between_visits_1m,6
34,cheque_count_6m_g40,7
128,response_viber,20
127,response_sms,26


In [15]:
(feature_importances['feature_importances'] == 0).sum()

141

In [16]:
train = train[feature_importances.features[feature_importances['feature_importances'] != 0].values]

In [17]:
train.shape

(687029, 27)

In [18]:
cols = train.columns
for i, col1 in enumerate(cols):
    for j, col2 in enumerate(cols[i:]):
        train[f'{col1}_mul_{col2}'] = train[col1]*train[col2]
        train[f'{col1}_div_{col2}'] = train[col1]/train[col2]

In [19]:
train.shape

(687029, 783)

In [20]:
pd.options.mode.use_inf_as_na = True

In [21]:
(train.isna().sum() > train.shape[0] / 2).sum()

178

In [22]:
index1 = train.isna().sum() < train.shape[0] / 2
train = train.loc[:, index1]
train.shape

(687029, 605)

In [23]:
model = LGBMClassifier(class_weight='balanced',  n_jobs=- 1, silent=True, n_estimators=35, max_depth=2)
model.fit(train, y)
feature_importances = pd.DataFrame({
    'features': train.columns,
    'feature_importances': model.feature_importances_}
            ).sort_values(by='feature_importances')
feature_importances.head()

,features,feature_importances
0,k_var_sku_price_6m_g26,0
398,cheque_count_12m_g46_mul_cheque_count_6m_g32,0
397,cheque_count_12m_g46_div_cheque_count_12m_g48,0
396,cheque_count_12m_g46_mul_cheque_count_12m_g48,0
395,cheque_count_12m_g46_div_cheque_count_12m_g46,0


In [24]:
feature_importances.tail()

,features,feature_importances
571,perdelta_days_between_visits_15_30d_div_respon...,4
598,cheque_count_6m_g40_div_response_sms,5
520,k_var_cheque_3m_div_response_sms,5
602,response_viber_div_response_sms,7
25,response_viber,17


In [25]:
(feature_importances['feature_importances'] == 0).sum()

556

In [26]:
train = train[feature_importances.features[feature_importances['feature_importances'] != 0].values]
train.shape

(687029, 49)

In [28]:
gc.collect()

122

In [95]:
%%time
initial_feauture = 'response_viber'
all_features = set(train.columns) - {'response_viber'}
dataset_now = train[[initial_feauture]]
best_score = 0
for t in range(6):
    k_fold = KFold(n_splits=2, shuffle=True)
    for train_index, test_index in k_fold.split(train.values):
        X_train, y_train = dataset_now.iloc[train_index], y.iloc[train_index]
        X_test, y_test = dataset_now.iloc[test_index], y.iloc[test_index]
        treatment_test = pdf.group.iloc[test_index].map({1: 'test', 0: 'control'})
        attr_test = pdf.response_att.iloc[test_index]
        seed = np.random.choice(1000000000)
        model = LGBMClassifier(class_weight='balanced',  n_jobs=-1, silent=True, n_estimators=45,
                               random_state=seed, max_depth=2)
        model.fit(X_train, y_train)
        ans = pd.DataFrame({
            'uplift': model.predict_proba(X_test)[:, 1],
            'response_att': attr_test,
            'group': treatment_test
        })
        best_score += custom_metric(ans)
best_score /= 12
best_score

CPU times: user 1min 5s, sys: 51.8 ms, total: 1min 5s
Wall time: 5.23 s


0.9902938807811595

In [96]:
def score_ds(columns_now):
    dataset_now = train[columns_now]
    score = 0
    for t in range(6):
        k_fold = KFold(n_splits=2, shuffle=True)
        for train_index, test_index in k_fold.split(train.values):
            X_train, y_train = dataset_now.iloc[train_index], y.iloc[train_index]
            X_test, y_test = dataset_now.iloc[test_index], y.iloc[test_index]
            X_train = X_train[X_train.columns]
            treatment_test = pdf.group.iloc[test_index].map({1: 'test', 0: 'control'})
            attr_test = pdf.response_att.iloc[test_index]
            model = LGBMClassifier(class_weight='balanced',  n_jobs=- 1, silent=True, n_estimators=45,
                                   random_state=seed, max_depth=2)
            model.fit(X_train, y_train)
            ans = pd.DataFrame({
                'uplift': model.predict_proba(X_test)[:, 1],
                'response_att': attr_test,
                'group': treatment_test
            })
            score += custom_metric(ans)
    return score/12

In [97]:
experiment = Experiment(workspace=ws, name="feature_selection_experiment_mul_div")
run =  experiment.start_logging(snapshot_directory=None)
run.log('class_weight', 'balanced')
run.log('n_estimators', 45)
run.log('max_depth', 2)

In [98]:
run.log('response_viber', best_score)

In [99]:
features_now = [initial_feauture]
all_features = set(train.columns) - {'response_viber'} 
with tqdm_notebook(total=len(train.columns)) as pbar:
    while True:
        best_epoch_score = -1000000
        best_epoch_feature = -10000000
        for feature in all_features:
            score = score_ds(features_now + [feature])
            if score > best_epoch_score:
                best_epoch_score = score
                best_epoch_feature = feature
        if best_epoch_score > best_score:
            all_features -= {best_epoch_feature}
            features_now += [best_epoch_feature]
            best_score = best_epoch_score
            print(best_epoch_score, best_epoch_feature)
            pbar.update(1)
            run.log(best_epoch_feature, best_epoch_score)
        else:
            print('All features were selected')
            run.complete()
            break

5.4870804205648325 stdev_days_between_visits_15d_div_response_sms
5.69428759297312 cheque_count_12m_g48_div_response_sms
5.987736220897379 k_var_cheque_3m_mul_k_var_days_between_visits_1m
6.058972537878213 cheque_count_12m_g48_div_response_viber
6.095310276784324 cheque_count_6m_g38_mul_k_var_cheque_3m
6.11549013392536 k_var_days_between_visits_1m_div_response_sms
6.144344288943259 food_share_1m_mul_k_var_days_between_visits_1m
6.146795558859556 k_var_count_per_cheque_6m_g27_div_response_sms
All features were selected



In [100]:
for feature in features_now:
    if '_div_' in feature:
        col_1, col_2 = feature.split('_div_')
        pdf[feature] = pdf[col_1]/pdf[col_2]
    if '_mul_' in feature:
        col_1, col_2 = feature.split('_mul_')
        pdf[feature] = pdf[col_1]*pdf[col_2]

In [101]:
needed_cols = ['group', 'response_att', 'CardHolder']

In [107]:
local_path = './dataset_final.csv'
pdf[features_now + needed_cols].to_csv(local_path)

In [108]:
datastore = ws.get_default_datastore()
datastore.upload(src_dir='/home/hacker_1/notebooks', target_path='data',overwrite=True)


Uploading an estimated of 912 files
Uploading /home/hacker_1/notebooks/AzureML/.git/HEAD
Uploading /home/hacker_1/notebooks/AzureML/.git/config
Uploading /home/hacker_1/notebooks/AzureML/.git/description
Uploading /home/hacker_1/notebooks/AzureML/.git/hooks/applypatch-msg.sample
Uploading /home/hacker_1/notebooks/AzureML/.git/hooks/commit-msg.sample
Uploading /home/hacker_1/notebooks/AzureML/.git/hooks/fsmonitor-watchman.sample
Uploading /home/hacker_1/notebooks/AzureML/.git/hooks/post-update.sample
Uploading /home/hacker_1/notebooks/AzureML/.git/hooks/pre-applypatch.sample
Uploading /home/hacker_1/notebooks/AzureML/.git/hooks/pre-commit.sample
Uploading /home/hacker_1/notebooks/AzureML/.git/hooks/pre-push.sample
Uploading /home/hacker_1/notebooks/AzureML/.git/hooks/pre-rebase.sample
Uploading /home/hacker_1/notebooks/AzureML/.git/hooks/pre-receive.sample
Uploading /home/hacker_1/notebooks/AzureML/.git/hooks/prepare-commit-msg.sample
Uploading /home/hacker_1/notebooks/AzureML/.git/hook

Uploaded /home/hacker_1/notebooks/AzureML/contrib/RAPIDS/imgs/fef7.png, 66 files out of an estimated total of 912
Uploading /home/hacker_1/notebooks/AzureML/contrib/RAPIDS/imgs/target_creation.png
Uploaded /home/hacker_1/notebooks/AzureML/contrib/RAPIDS/imgs/fef2.png, 67 files out of an estimated total of 912
Uploading /home/hacker_1/notebooks/AzureML/contrib/RAPIDS/imgs/targeterror1.png
Uploaded /home/hacker_1/notebooks/AzureML/contrib/RAPIDS/imgs/fef6.png, 68 files out of an estimated total of 912
Uploading /home/hacker_1/notebooks/AzureML/contrib/RAPIDS/imgs/targeterror2.png
Uploaded /home/hacker_1/notebooks/AzureML/contrib/RAPIDS/imgs/CPUBase.png, 69 files out of an estimated total of 912
Uploading /home/hacker_1/notebooks/AzureML/contrib/RAPIDS/imgs/targetsuccess.png
Uploaded /home/hacker_1/notebooks/AzureML/.git/hooks/pre-rebase.sample, 70 files out of an estimated total of 912
Uploading /home/hacker_1/notebooks/AzureML/contrib/RAPIDS/imgs/training.png
Uploaded /home/hacker_1/not

Uploaded /home/hacker_1/notebooks/AzureML/how-to-use-azureml/automated-machine-learning/classification-text-dnn/helper.py, 147 files out of an estimated total of 912
Uploaded /home/hacker_1/notebooks/AzureML/how-to-use-azureml/automated-machine-learning/forecasting-beer-remote/infer.py, 148 files out of an estimated total of 912
Uploading /home/hacker_1/notebooks/AzureML/how-to-use-azureml/azure-databricks/databricks-as-remote-compute-target/testdata.txt
Uploaded /home/hacker_1/notebooks/AzureML/how-to-use-azureml/automated-machine-learning/forecasting-bike-share/run_forecast.py, 149 files out of an estimated total of 912
Uploading /home/hacker_1/notebooks/AzureML/how-to-use-azureml/azure-databricks/databricks-as-remote-compute-target/train-db-dbfs.py
Uploaded /home/hacker_1/notebooks/AzureML/how-to-use-azureml/automated-machine-learning/forecasting-energy-demand/auto-ml-forecasting-energy-demand.yml, 150 files out of an estimated total of 912
Uploading /home/hacker_1/notebooks/AzureML

Uploaded /home/hacker_1/notebooks/AzureML/how-to-use-azureml/deployment/onnx/mnist.py, 248 files out of an estimated total of 912
Uploading /home/hacker_1/notebooks/AzureML/how-to-use-azureml/explain-model/azure-integration/scoring-time/score_local_explain.py
Uploaded /home/hacker_1/notebooks/AzureML/how-to-use-azureml/deployment/onnx/onnx-train-pytorch-aml-deploy-mnist.ipynb, 249 files out of an estimated total of 912
Uploaded /home/hacker_1/notebooks/AzureML/how-to-use-azureml/deployment/production-deploy-to-aks/sklearn_regression_model.pkl, 250 files out of an estimated total of 912
Uploaded /home/hacker_1/notebooks/AzureML/how-to-use-azureml/deployment/spark/iris.model/data/part-00000-dabcf097-2b45-4b28-bbca-6c17889ddcbf-c000.snappy.parquet, 251 files out of an estimated total of 912
Uploading /home/hacker_1/notebooks/AzureML/how-to-use-azureml/explain-model/azure-integration/scoring-time/score_local_explain_keras.py
Uploaded /home/hacker_1/notebooks/AzureML/how-to-use-azureml/depl

Uploaded /home/hacker_1/notebooks/AzureML/how-to-use-azureml/machine-learning-pipelines/intro-to-pipelines/testdata.txt, 327 files out of an estimated total of 912
Uploading /home/hacker_1/notebooks/AzureML/how-to-use-azureml/manage-azureml-service/authentication-in-azureml/authentication-in-azureml.ipynb
Uploaded /home/hacker_1/notebooks/AzureML/how-to-use-azureml/machine-learning-pipelines/intro-to-pipelines/utils.py, 328 files out of an estimated total of 912
Uploading /home/hacker_1/notebooks/AzureML/how-to-use-azureml/manage-azureml-service/authentication-in-azureml/images/svc-pr-1.PNG
Uploaded /home/hacker_1/notebooks/AzureML/how-to-use-azureml/machine-learning-pipelines/nyc-taxi-data-regression-model-building/scripts/prepdata/transform.py, 329 files out of an estimated total of 912
Uploading /home/hacker_1/notebooks/AzureML/how-to-use-azureml/manage-azureml-service/authentication-in-azureml/images/svc-pr-2.PNG
Uploaded /home/hacker_1/notebooks/AzureML/how-to-use-azureml/machine-

...

Uploaded /home/hacker_1/notebooks/C:/Users/123/Downloads/part-00025, 698 files out of an estimated total of 912
Uploading /home/hacker_1/notebooks/catboost/tutorials/classification/README.md
Uploaded /home/hacker_1/notebooks/C:/Users/123/Downloads/part-00024, 699 files out of an estimated total of 912
Uploading /home/hacker_1/notebooks/catboost/tutorials/classification/classification_tutorial.ipynb
Uploaded /home/hacker_1/notebooks/catboost/tutorials/apply_model/rust/readme.md, 700 files out of an estimated total of 912
Uploading /home/hacker_1/notebooks/catboost/tutorials/classification/classification_with_parameter_tuning_tutorial.ipynb
Uploaded /home/hacker_1/notebooks/C:/Users/123/Downloads/part-00011, 701 files out of an estimated total of 912
Uploaded /home/hacker_1/notebooks/catboost/tutorials/apply_model/rust/train_model.ipynb, 702 files out of an estimated total of 912
Uploading /home/hacker_1/notebooks/catboost/tutorials/cmdline_tutorial.md
Uploaded /home/hacker_1/notebooks/C

Uploaded /home/hacker_1/notebooks/h2o/python/EEG_eyestate_sklearn_NOPASS.ipynb, 762 files out of an estimated total of 912
Uploaded /home/hacker_1/notebooks/h2o/python/H2O_chicago_crimes.ipynb, 763 files out of an estimated total of 912
Uploading /home/hacker_1/notebooks/h2o/python/README.md
Uploading /home/hacker_1/notebooks/h2o/python/airlines_demo_small.ipynb
Uploaded /home/hacker_1/notebooks/h2o/python/H2O_tutorial_breast_cancer_classification.ipynb, 764 files out of an estimated total of 912
Uploading /home/hacker_1/notebooks/h2o/python/citi_bike_large.ipynb
Uploaded /home/hacker_1/notebooks/h2o/python/H2O_tutorial_eeg_eyestate.ipynb, 765 files out of an estimated total of 912
Uploading /home/hacker_1/notebooks/h2o/python/citi_bike_small.ipynb
Uploaded /home/hacker_1/notebooks/h2o/python/airlines_demo_small.ipynb, 766 files out of an estimated total of 912
Uploading /home/hacker_1/notebooks/h2o/python/cm_roc.ipynb
Uploaded /home/hacker_1/notebooks/h2o/python/README.md, 767 files o

Uploaded /home/hacker_1/notebooks/pytorch/beginner/hybrid_frontend/learning_hybrid_frontend_through_example_tutorial.ipynb, 847 files out of an estimated total of 912
Uploading /home/hacker_1/notebooks/pytorch/beginner/nlp/pytorch_tutorial.ipynb
Uploaded /home/hacker_1/notebooks/pytorch/beginner/former_torchies/parallelism_tutorial.ipynb, 848 files out of an estimated total of 912
Uploading /home/hacker_1/notebooks/pytorch/beginner/nlp/sequence_models_tutorial.ipynb
Uploaded /home/hacker_1/notebooks/h2o/python/custom_loss_function_demo.ipynb, 849 files out of an estimated total of 912
Uploading /home/hacker_1/notebooks/pytorch/beginner/nlp/word_embeddings_tutorial.ipynb
Uploaded /home/hacker_1/notebooks/pytorch/beginner/former_torchies/autograd_tutorial_old.ipynb, 850 files out of an estimated total of 912
Uploading /home/hacker_1/notebooks/pytorch/beginner/nn_tutorial.ipynb
Uploaded /home/hacker_1/notebooks/pytorch/beginner/examples_tensor/two_layer_net_tensor.ipynb, 851 files out of 

Uploaded /home/hacker_1/notebooks/h2o/python/turbofan_NOPASS_phm.ipynb, 894 files out of an estimated total of 912
Uploaded /home/hacker_1/notebooks/catboost/tutorials/python_tutorial_with_tasks.ipynb, 895 files out of an estimated total of 912
Uploaded /home/hacker_1/notebooks/catboost/tutorials/ru/ml_session_2018_tutorial_ru.ipynb, 896 files out of an estimated total of 912
Uploaded /home/hacker_1/notebooks/catboost/tutorials/classification/classification_tutorial.ipynb, 897 files out of an estimated total of 912
Uploaded /home/hacker_1/notebooks/catboost/tutorials/feature_selection/eval_tutorial.ipynb, 898 files out of an estimated total of 912
Uploaded /home/hacker_1/notebooks/catboost/tutorials/events/2019_odsc_east/odsc_east_2019.ipynb, 899 files out of an estimated total of 912
Uploaded /home/hacker_1/notebooks/catboost/tutorials/events/pydata_moscow_oct_13_2018.ipynb, 900 files out of an estimated total of 912
Uploaded /home/hacker_1/notebooks/catboost/tutorials/events/datastar

$AZUREML_DATAREFERENCE_1f466aefde534767b8c96cfb5b689c9b

In [110]:
dataset = Dataset.Tabular.from_delimited_files(datastore.path(f'data/dataset_final.csv'))

registerered_ds = dataset.register(workspace=ws,
                                 name=f'dataset_final',
                                 description=f'features_final',
                                 tags={'CSV':'True'}, create_new_version=True)

In [111]:
pdf.shape

(687029, 216)